In [1]:
import numpy as np
import numpy.linalg as lia
import pandas as pd
import matplotlib as plt
import cv2 as cv
from sklearn import svm 
from sklearn import datasets
from sklearn.datasets import fetch_openml
from sklearn.naive_bayes import GaussianNB

In [2]:
digits = datasets.load_digits()
wine = fetch_openml(name='wine', version=1)

In [3]:
# normalization of digits and wine data

digits_data_norm = []

for col in digits.data:
    col_norm = col/np.max(col)
    digits_data_norm.append(col_norm)

digits.data = np.asarray(digits_data_norm)

wine_data_norm = []

for col in wine.data.T:
    col_norm = col/np.amax(col)
    wine_data_norm.append(col_norm)
    
wine.data = np.asarray(wine_data_norm).T

In [4]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index+(foldIndex*digitsValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []
    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index+(foldIndex*wineValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []
    
    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)

In [5]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [6]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [7]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=0.5, momentum=0.9, max_termination=10, max_iters=1000):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_termination = max_termination
        self.max_iters = max_iters
        self.deltas = []
        
    def run(self, gradient_fn, x, y, w):
        t = 1
        
        min_cost = np.inf
        termination_count = 0        
        weight_history = []
        error_history = []
                
        for number_of_targets in range(len(y[0])):
            weight_history.append([])
        
        while termination_count < self.max_termination and t < self.max_iters:
            gradients = gradient_fn(x, y, w, self.batch_size)   
            
            for c in range(len(y[0])):
                if(t==1):
                    w[c] = w[c] - self.learning_rate * gradients[c]
                else:
                    delta_w = (self.momentum)*(self.deltas[-(len(y[0]))]) + (1-self.momentum)*gradients[c]
                    w[c] = w[c] - (self.learning_rate)*(delta_w)
                self.deltas.append(w[c])
            
            a = np.asarray(x)
            b = np.asarray(w)
    
            yh=[]
            for i, x_c in enumerate(a):
                yh_x=[]

                for c in range(len(b)):
                    w_x =  b[c] @ x_c
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  b[i] @ x_c
                        den += np.exp(w_x)

                    yh_c = num/den
                    yh_x.append(yh_c)
                    
                yh.append(yh_x)
                
            step_cost = 0
                
            def cost(yh, y):
                return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))
                
            for sample_index, yh_x in enumerate(yh):
                c = np.argmax(y[sample_index])
                cst = cost(yh_x[c], y[sample_index][c])
                step_cost += cst
            
            for c in range(len(b)):
                weight_history[c].append(w[c])

            error_history.append(step_cost)
            
            # We found an alternate termination condition that terminates faster as
            # the suggested condition would run for a longtime for us (~1hr).
            
            # We track the lowest step_cost encountered, and if the 
            # next max_termination-number of steps do not have a better cost, then
            # it terminates.

            if step_cost < min_cost:
                min_cost = step_cost
                termination_count = 0
                print(f"\t\tStep {t}: new best cost of {min_cost:.3f}")
            else:
                termination_count += 1
                print(f"\t\tStep {t}")
            
            t += 1
        
        # take the weight prior to the last max_termination-number of weights
        # (as it is guaranteed to be the best prior to termination).
        
        index_best = len(error_history)-self.max_termination-1
        
        w_best = []
        
        for c in range(len(y[0])):
            w_best.append(weight_history[c][index_best])
            
        #return w_best
        
        return w

In [8]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    gradients[c] += cost_c

            return gradients
        
        if self.add_bias:
            x = np.asarray(x)
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
        return self
    
    def predict(self, x):
        if self.add_bias:
            x = np.asarray(x)
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])

        a = np.asarray(x)
        b = np.asarray(self.w)

        yh=[]
        
        for i, x_c in enumerate(a):
            yh_x=[]
            
            for c in range(len(b)):
                w_x =  b[c] @ x_c
                num = np.exp(w_x)

                den = 0
                for i in range(len(b)):
                    w_x =  b[i] @ x_c
                    den += np.exp(w_x)

                yh_c = num/den
                yh_x.append(yh_c)
                
            yh.append(yh_x)
        
        return yh

In [ ]:
def accurate(a, b):
    return np.argmax(a) == np.argmax(b)

def cost(yh, y):
    return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))

# TODO: grid-search to find lowest cost combination of model hyper-parameters

batch_size = 30
learning_rate = 0.04
momentum = 0.2

print("Model hyper-parameters:")
print("\tMini-batch size:", batch_size)
print("\tLearning rate:", learning_rate)
print("\tMomentum:", momentum)
print("\n")

digits_training_accuracy = 0
digits_training_cost = 0
digits_validation_accuracy = 0
digits_validation_cost = 0

print("Digits gradient descent:")

for fold_index, fold in enumerate(xDigitsTrainingSets):
    print(f"\tCross-validation fold {fold_index+1}")
    
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(add_bias=True)
    
    logisticRegressionModel.fit(fold, yDigitsTrainingSets[fold_index], gradientDescentModel)
    yh_training = logisticRegressionModel.predict(xDigitsTrainingSets[fold_index])
    yh_validation = logisticRegressionModel.predict(xDigitsValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh_training):
        if accurate(yh_x, yDigitsTrainingSets[fold_index][sample_index]):
            digits_training_accuracy += 1
        c = np.argmax(yDigitsTrainingSets[fold_index][sample_index])
        cst = cost(yh_x[c], yDigitsTrainingSets[fold_index][sample_index][c])
        digits_training_cost += cst
    
    for sample_index, yh_x in enumerate(yh_validation):
        if accurate(yh_x, yDigitsValidationSets[fold_index][sample_index]):
            digits_validation_accuracy += 1
        c = np.argmax(yDigitsValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yDigitsValidationSets[fold_index][sample_index][c])
        digits_validation_cost += cst
        
digits_training_accuracy /= 4*len(digits.data)
digits_training_cost /= 4
digits_validation_accuracy /= len(digits.data)

wine_training_accuracy = 0
wine_training_cost = 0
wine_validation_accuracy = 0
wine_validation_cost = 0

print("Wine gradient descent:")

for fold_index, fold in enumerate(xWineTrainingSets):
    print(f"\tCross-validation fold {fold_index+1}")
    
    gradientDescentModel = GradientDescent(batch_size, learning_rate, momentum)
    logisticRegressionModel = LogisticRegression(add_bias=True)
    
    logisticRegressionModel.fit(fold, yWineTrainingSets[fold_index], gradientDescentModel)
    yh_training = logisticRegressionModel.predict(xWineTrainingSets[fold_index])
    yh_validation = logisticRegressionModel.predict(xWineValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh_training):
        if accurate(yh_x, yWineTrainingSets[fold_index][sample_index]):
            wine_training_accuracy += 1
        c = np.argmax(yWineTrainingSets[fold_index][sample_index])
        cst = cost(yh_x[c], yWineTrainingSets[fold_index][sample_index][c])
        wine_training_cost += cst
    
    for sample_index, yh_x in enumerate(yh_validation):
        if accurate(yh_x, yWineValidationSets[fold_index][sample_index]):
            wine_validation_accuracy += 1
        c = np.argmax(yWineValidationSets[fold_index][sample_index])
        cst = cost(yh_x[c], yWineValidationSets[fold_index][sample_index][c])
        wine_validation_cost += cst

wine_training_accuracy /= 4*len(wine.data)
wine_training_cost /= 4
wine_validation_accuracy /= len(wine.data)
    
print("\n")
print(f"Digits training accuracy: {digits_training_accuracy*100:.1f}%")
print(f"Digits training cost: {digits_training_cost:.3f}")
print(f"Digits validation accuracy: {digits_validation_accuracy*100:.1f}%")
print(f"Digits validation cost: {digits_validation_cost:.3f}")
print(f"Wine training accuracy: {wine_training_accuracy*100:.1f}%")
print(f"Wine training cost: {wine_training_cost:.3f}")
print(f"Wine validation accuracy: {wine_validation_accuracy*100:.1f}%")
print(f"Wine validation cost: {wine_validation_cost:.3f}")


Model hyper-parameters:
	Mini-batch size: 30
	Learning rate: 0.04
	Momentum: 0.2


Digits gradient descent:
	Cross-validation fold 1
		Step 1: new best cost of 1838.674
		Step 2
		Step 3: new best cost of 1803.062
		Step 4: new best cost of 1780.246
		Step 5: new best cost of 1755.715
		Step 6: new best cost of 1706.679
		Step 7: new best cost of 1700.851
		Step 8: new best cost of 1695.124
		Step 9: new best cost of 1634.909
		Step 10: new best cost of 1588.966
		Step 11
		Step 12: new best cost of 1552.115
		Step 13: new best cost of 1547.302
		Step 14: new best cost of 1504.241
		Step 15: new best cost of 1480.087
		Step 16: new best cost of 1434.071
		Step 17: new best cost of 1422.272
		Step 18: new best cost of 1416.952
		Step 19: new best cost of 1407.404
		Step 20
		Step 21: new best cost of 1365.877
		Step 22: new best cost of 1357.680
		Step 23
		Step 24: new best cost of 1332.265
		Step 25: new best cost of 1307.595
		Step 26: new best cost of 1297.617
		Step 27
		Step 28: n

		Step 18: new best cost of 1412.377
		Step 19
		Step 20: new best cost of 1379.277
		Step 21
		Step 22: new best cost of 1376.610
		Step 23: new best cost of 1342.424
		Step 24
		Step 25: new best cost of 1309.832
		Step 26
		Step 27
		Step 28: new best cost of 1289.235
		Step 29: new best cost of 1266.592
		Step 30: new best cost of 1264.644
		Step 31: new best cost of 1249.703
		Step 32: new best cost of 1236.879
		Step 33
		Step 34: new best cost of 1233.030
		Step 35
		Step 36: new best cost of 1226.815
		Step 37
		Step 38: new best cost of 1204.525
		Step 39: new best cost of 1197.577
		Step 40: new best cost of 1174.815
		Step 41: new best cost of 1173.543
		Step 42
		Step 43: new best cost of 1161.227
		Step 44
		Step 45: new best cost of 1151.482
		Step 46
		Step 47: new best cost of 1149.104
		Step 48: new best cost of 1137.321
		Step 49
		Step 50
		Step 51
		Step 52
		Step 53
		Step 54: new best cost of 1137.159
		Step 55
		Step 56: new best cost of 1131.053
		Step 57: new b

		Step 89
		Step 90
		Step 91
		Step 92
		Step 93
		Step 94: new best cost of 100.345
		Step 95
		Step 96
		Step 97
		Step 98: new best cost of 100.030
		Step 99
		Step 100: new best cost of 99.502
		Step 101
		Step 102
		Step 103
		Step 104
		Step 105: new best cost of 99.014
		Step 106
		Step 107
		Step 108: new best cost of 99.009
		Step 109
		Step 110
		Step 111
		Step 112
		Step 113
		Step 114
		Step 115: new best cost of 98.358
		Step 116
		Step 117
		Step 118
		Step 119
		Step 120
		Step 121: new best cost of 97.876
		Step 122
		Step 123
		Step 124
		Step 125
		Step 126: new best cost of 97.688
		Step 127
		Step 128
		Step 129
		Step 130
		Step 131
		Step 132
		Step 133: new best cost of 97.212
		Step 134
		Step 135
		Step 136
		Step 137
		Step 138
		Step 139
		Step 140: new best cost of 97.204
		Step 141: new best cost of 96.999
		Step 142
		Step 143
		Step 144
		Step 145: new best cost of 96.713
		Step 146: new best cost of 96.660
		Step 147
		Step 148
		Step 149
		Step 150
		

In [ ]:
euclidean = lambda x1, x2: np.sqrt(np.sum((x1 - x2)**2, axis=-1))
manhattan = lambda x1, x2: np.sum(np.abs(x1 - x2), axis=-1)

class KNN:

    def __init__(self, K=1, dist_fn= euclidean):
        self.dist_fn = dist_fn
        self.K = K
        return
    
    def fit(self, x, y):
        self.x = x
        self.y = y
        self.C = len(y[0])
        return self
    
    def predict(self, x_test):
        num_test = x_test.shape[0]
        distances = self.dist_fn(self.x[None,:,:], x_test[:,None,:])
        knns = np.zeros((num_test, self.K), dtype=int)
        y_prob = np.zeros((num_test),dtype=int)
        counts = np.zeros((num_test, self.C))
        
        for i in range(num_test):
            knns[i,:] = np.argsort(distances[i])[:self.K]
            k_count=np.zeros(self.K, dtype=int)
            
            for s, arr in enumerate(self.y[knns[i,:]]):
                k_count[s] = np.argmax(arr)
            
            y_prob_i, counts_i = np.unique(k_count, return_counts=True)
            y_prob[i] = int(y_prob_i[np.argmax(counts_i)])
        
        return y_prob, knns

In [ ]:
KNNmodel = KNN(K=11)

digits_knn_accuracy = 0

for fold in range(5):
    y_prob, knns = KNNmodel.fit(np.asarray(xDigitsTrainingSets[fold]), np.asarray(yDigitsTrainingSets[fold])).predict(np.asarray(xDigitsValidationSets[fold]))
    
    for i, prob in enumerate(y_prob):
        if prob == np.argmax(yDigitsValidationSets[fold][i]):
            digits_knn_accuracy += 1

digits_knn_accuracy /= len(digits.data)

print(f"KNN digits validation accuracy: {digits_knn_accuracy*100:.1f}%")

KNNmodel = KNN(K=7)

wine_knn_accuracy = 0

for fold in range(5):
    y_prob, knns = KNNmodel.fit(np.asarray(xWineTrainingSets[fold]), np.asarray(yWineTrainingSets[fold])).predict(np.asarray(xWineValidationSets[fold]))
    
    for i, prob in enumerate(y_prob):
        if prob == np.argmax(yWineValidationSets[fold][i]):
            wine_knn_accuracy += 1
            
wine_knn_accuracy /= len(wine.data)

print(f"KNN wine validation accuracy: {wine_knn_accuracy*100:.1f}%")

In [ ]:
def HoGFeatures(img, cellSize, blockSize, nbins):
    cell_size = (cellSize, cellSize)
    block_size = (blockSize, blockSize)
    
    hog = cv.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                     img.shape[0] // cell_size[0] * cell_size[0]),
                           _blockSize=(block_size[1] * cell_size[1],
                                       block_size[0] * cell_size[0]),
                           _blockStride=(cell_size[1], cell_size[0]),
                           _cellSize=(cell_size[1], cell_size[0]),
                           _nbins=nbins
    )
    
    return hog

In [ ]:
def makeHoGFeatures(imageArray):
    HoG = HoGFeatures(imageArray[0], 2, 2, 2)
    features = []
    
    for i, image in enumerate(imageArray):
        features.append(HoG.compute((image*255).astype(np.uint8)))
        
    features = np.array(np.squeeze(features))
    
    return features

In [ ]:
digits_svc_accuracy = 0

for fold in range(5):
    numbers_training = []
    numbers_validation = []
    
    for i, number in enumerate(xDigitsTrainingSets[fold]):
        numbers_training.append(np.asarray(number).reshape(8, 8))
        
    for i, number in enumerate(xDigitsValidationSets[fold]):
        numbers_validation.append(np.asarray(number).reshape(8, 8))  
        
    HoGs_training = makeHoGFeatures(np.asarray(numbers_training))
    HoGs_validation = makeHoGFeatures(np.asarray(numbers_validation))

    clf = svm.SVC(gamma='auto', C=100) 
    
    labels_training = np.zeros(len(yDigitsTrainingSets[fold]))
    
    for i, arr in enumerate(yDigitsTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yDigitsValidationSets[fold]))
    
    for i, arr in enumerate(yDigitsValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    clf.fit(HoGs_training, labels_training)

    labels_predicted = clf.predict(HoGs_validation)
    
    for i, label in enumerate(labels_predicted):
        if label == labels_validation[i]:
            digits_svc_accuracy += 1

digits_svc_accuracy /= len(digits.data)

print(f"SVC digits validation accuracy: {digits_svc_accuracy*100:.1f}%")

In [ ]:
digits_naive_accuracy = 0

for fold in range(5):
    labels_training = np.zeros(len(yDigitsTrainingSets[fold]))
    
    for i, arr in enumerate(yDigitsTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yDigitsValidationSets[fold]))
    
    for i, arr in enumerate(yDigitsValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    gnb = GaussianNB()
    y_pred = gnb.fit(np.asarray(xDigitsTrainingSets[fold]), labels_training).predict(np.asarray(xDigitsValidationSets[fold]))

    for i, label in enumerate(y_pred):
        if label == labels_validation[i]:
            digits_naive_accuracy += 1

digits_naive_accuracy /= len(digits.data)

print(f"Naive base digits validation accuracy: {digits_naive_accuracy*100:.1f}%\n")

wine_naive_accuracy = 0

for fold in range(5):
    labels_training = np.zeros(len(yWineTrainingSets[fold]))
    
    for i, arr in enumerate(yWineTrainingSets[fold]):
        labels_training[i] = np.argmax(arr)
        
    labels_validation = np.zeros(len(yWineValidationSets[fold]))
    
    for i, arr in enumerate(yWineValidationSets[fold]):
        labels_validation[i] = np.argmax(arr)
    
    gnb = GaussianNB()
    y_pred = gnb.fit(np.asarray(xWineTrainingSets[fold]), labels_training).predict(np.asarray(xWineValidationSets[fold]))

    for i, label in enumerate(y_pred):
        if label == labels_validation[i]:
            wine_naive_accuracy += 1

wine_naive_accuracy /= len(wine.data)

print(f"Naive base wine validation accuracy: {wine_naive_accuracy*100:.1f}%")